In [1]:
%pip install pandas matplotlib seaborn plotly dash scikit-learn joblib
import pandas as pd
from pptx import Presentation
from pptx.util import Inches, Pt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly.express as px
import dash
from dash import dcc, html, Input, Output
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
import re

from io import BytesIO
import base64
import plotly.io as pio

Note: you may need to restart the kernel to use updated packages.


In [2]:
accidents = pd.read_csv("cleaned_data/accidents.csv")
path = "report/dataset2/"

In [3]:
accidents['date'] = pd.to_datetime(accidents['date'], errors='coerce')
accidents['Year-Month'] = accidents['date'].dt.to_period('M').astype(str)

unique_industries = accidents['industry_sector_description'].dropna().unique()
unique_accident_categories = accidents['accident_category_description'].dropna().unique()

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Workplace Accident Analysis Dashboard", style={'textAlign': 'center'}),
    html.Label("Where (Industry):"),
    dcc.Dropdown(
        id='industry-dropdown',
        options=[{'label': i, 'value': i} for i in unique_industries],
        multi=True
    ),
    html.Label("How (Accident Category):"),
    dcc.Dropdown(
        id='accident-dropdown',
        options=[{'label': i, 'value': i} for i in unique_accident_categories],
        multi=True
    ),
    dcc.Graph(id='accident-trends-graph'),
    dcc.Graph(id='accident-source-bar'),
    html.Button("Export to PowerPoint", id='export-button', n_clicks=0),
    dcc.Download(id="download-ppt")
], style={'width': '48%', 'display': 'inline-block'})

@app.callback(
    [Output('accident-trends-graph', 'figure'),
     Output('accident-source-bar', 'figure')],
    [Input('industry-dropdown', 'value'),
     Input('accident-dropdown', 'value')]
)
def update_graphs(selected_industries, selected_accidents):
    filtered_data = accidents.copy()
    if selected_industries:
        filtered_data = filtered_data[filtered_data['industry_sector_description'].isin(selected_industries)]
    if selected_accidents:
        filtered_data = filtered_data[filtered_data['accident_category_description'].isin(selected_accidents)]

    accident_trends_filtered = filtered_data.groupby('Year-Month').size().reset_index(name='Accident Count')
    fig1 = px.line(accident_trends_filtered, x='Year-Month', y='Accident Count', title='Monthly Workplace Accident Trends', markers=True)

    accident_sources_filtered = filtered_data['accident_source_category_description1'].value_counts().reset_index()
    accident_sources_filtered.columns = ['Accident Source', 'Count']
    fig2 = px.bar(accident_sources_filtered, x='Accident Source', y='Count', title='Accident Sources Distribution', text_auto=True)

    return fig1, fig2

@app.callback(
    # Output("download-ppt", "data"),
    [Input("export-button", "n_clicks"),
     Input('accident-trends-graph', 'figure'),
     Input('accident-source-bar', 'figure')]
)
def export_to_ppt(n_clicks, fig1, fig2):
    if n_clicks > 0:
        ppt = Presentation()
        slide_layout = ppt.slide_layouts[5]

        # slide with trends graph
        slide = ppt.slides.add_slide(slide_layout)
        slide.shapes.title.text = "Monthly Workplace Accident Trends"
        image_stream = BytesIO()
        pio.write_image(fig1, image_stream, format='png')
        image_stream.seek(0)
        slide.shapes.add_picture(image_stream, Inches(1), Inches(1), Inches(7), Inches(5))

        # slide with sources bar graph
        slide = ppt.slides.add_slide(slide_layout)
        slide.shapes.title.text = "Accident Sources Distribution"
        image_stream = BytesIO()
        pio.write_image(fig2, image_stream, format='png')
        image_stream.seek(0)
        slide.shapes.add_picture(image_stream, Inches(1), Inches(1), Inches(7), Inches(5))

        ppt_file = path+"interactive_plt.pptx"#BytesIO()
        ppt.save(ppt_file)
        print("Your powerpoint slides saved in", ppt_file)
        # ppt_file.seek(0)
        # temp = send_file(ppt_file, download_name="Accident_Report.pptx", as_attachment=True)
        # print('***')
        # print(ppt_file)

        # return send_file(ppt_file, download_name="Accident_Report.pptx", as_attachment=True)

if __name__ == '__main__':
    app.run_server(debug=True)
